In [1]:
import sys
import subprocess
import pkg_resources
import os
import pandas as pd
sys.path.append(os.path.abspath('../scripts'))

from myFunctions import install_packages, save_table 
install_packages()


Installing required packages: ['numpy', 'pandas', 'scikit-learn', 'joblib', 'pyarrow', 'fastparquet', 'plotly', 'matplotlib', 'seaborn', 'MetaTrader5', 'tabulate', 'optuna', 'torch', 'tqdm', 'shap', 'kaleido', 'statsmodels', 're']
numpy is already installed.
pandas is already installed.
scikit-learn is already installed.
joblib is already installed.
pyarrow is already installed.
fastparquet is already installed.
plotly is already installed.
matplotlib is already installed.
Installing seaborn...
MetaTrader5 is already installed.
tabulate is already installed.
optuna is already installed.
torch is already installed.
tqdm is already installed.
shap is already installed.
kaleido is already installed.
Installing statsmodels...
Installing re...
Error installing re: Command '['c:\\Users\\guitz\\anaconda3\\envs\\pytorch_env\\python.exe', '-m', 'pip', 'install', 're']' returned non-zero exit status 1.
All packages are verified.


In [2]:
input_target = os.path.join('..', 'data', 'target')
input_features = os.path.join('..', 'data', 'features')
output_dir = os.path.join('..', 'data', 'processed_data')


In [3]:
df_features = pd.read_parquet(os.path.join(input_features, 'features.parquet'))
df_daily_target = pd.read_parquet(os.path.join(input_target, 'daily_target.parquet'))
df_timestamp_target = pd.read_parquet(os.path.join(input_target, 'timestamp_target.parquet'))
df_daily = df_features.copy()
df_timestamp = df_features.copy()


In [4]:
print(df_timestamp.shape)
print(df_timestamp_target.shape)
df_timestamp = df_timestamp.merge(
    df_timestamp_target[['time', 'close_price_target', 'open_price_target', 'behavior_target']],
    on='time',
    how='inner'
)
print(df_timestamp.shape)

(22356, 169)
(22356, 6)
(22356, 172)


In [5]:
df_timestamp_target.isna().sum()

time                  0
open_BGI$             0
close_BGI$            0
close_price_target    1
open_price_target     1
behavior_target       0
dtype: int64

In [6]:
df_timestamp.dropna(inplace=True)

In [7]:
df_daily.head()

,time,open_AGFS,high_AGFS,low_AGFS,close_AGFS,tick_volume_AGFS,spread_AGFS,real_volume_AGFS,open_BGI$,high_BGI$,...,EMA55_CCM$,EMA9_GOLD11,EMA21_GOLD11,EMA55_GOLD11,EMA9_IBOV,EMA21_IBOV,EMA55_IBOV,EMA9_DOL$,EMA21_DOL$,EMA55_DOL$
96,2022-06-02 09:00:00,1767.0,1767.0,1763.0,1763.0,0.0,0.0,0.0,320.52,321.21,...,122.110865,9.447520,9.432993,9.380451,111438.765770,111480.083554,111455.854537,5597.257017,5589.642700,5564.905769
97,2022-06-02 09:15:00,1767.0,1767.0,1763.0,1763.0,0.0,0.0,0.0,319.73,319.73,...,122.120834,9.448016,9.434539,9.382935,111423.012616,111469.166867,111452.431161,5593.956413,5588.834637,5565.471777
98,2022-06-02 09:30:00,1767.0,1767.0,1763.0,1763.0,0.0,0.0,0.0,319.73,319.73,...,122.120090,9.448413,9.435945,9.385330,111410.410093,111459.242606,111449.130048,5588.428931,5586.787761,5565.502035
99,2022-06-02 09:45:00,1767.0,1767.0,1763.0,1763.0,0.0,0.0,0.0,319.98,319.98,...,122.100087,9.448730,9.437222,9.387640,111400.328074,111450.220551,111445.946832,5584.584345,5585.189419,5565.634320
100,2022-06-02 10:00:00,1764.0,1778.0,1763.0,1777.0,9986.0,0.0,6702403.0,319.39,320.57,...,122.083655,9.446984,9.437475,9.389510,111577.062459,111526.018683,111475.877302,5583.240876,5584.523744,5566.071201


In [8]:
def pivot_data(df, target=None):
    """
    Pivot a DataFrame by hour for each day, excluding columns containing the target keyword.

    Parameters:
    df (pd.DataFrame): Input DataFrame with a 'time' column containing datetime values.
    target (str, optional): Keyword to exclude columns containing this value from the pivot. Default is None.

    Returns:
    pd.DataFrame: Pivoted DataFrame with columns for each hour of the day and variables suffixed with '_hhmm'.

    Example:
    >>> data = {
    ...     'time': ['2022-06-02 09:00:00', '2022-06-02 09:15:00', '2022-06-02 09:30:00'],
    ...     'open_BGI$': [313.94, 313.17, 312.85],
    ...     'close_BGI$': [313.75, 312.73, 313.00],
    ... }
    >>> df = pd.DataFrame(data)
    >>> df['time'] = pd.to_datetime(df['time'])
    >>> print(pivoted)
    date       open_BGI$_0900  open_BGI$_0915  open_BGI$_0930  close_BGI$_0900  close_BGI$_0915  close_BGI$_0930
    2022-06-02        313.94          313.17          312.85           313.75          312.73          313.00
    """
    df['date'] = pd.to_datetime(df['time']).dt.date
    df['hour'] = pd.to_datetime(df['time']).dt.strftime('%H%M')

    columns_to_pivot = [col for col in df.columns if col not in ['time', 'date', 'hour']]
    pivoted = df.pivot(index='date', columns='hour', values=columns_to_pivot)
    pivoted.columns = [f"{col}_{hour}" for col, hour in pivoted.columns]

    pivoted.reset_index(inplace=True)
    return pivoted


In [9]:
df_daily.columns

Index(['time', 'open_AGFS', 'high_AGFS', 'low_AGFS', 'close_AGFS',
       'tick_volume_AGFS', 'spread_AGFS', 'real_volume_AGFS', 'open_BGI$',
       'high_BGI$',
       ...
       'EMA55_CCM$', 'EMA9_GOLD11', 'EMA21_GOLD11', 'EMA55_GOLD11',
       'EMA9_IBOV', 'EMA21_IBOV', 'EMA55_IBOV', 'EMA9_DOL$', 'EMA21_DOL$',
       'EMA55_DOL$'],
      dtype='object', length=169)

In [10]:
df_daily =  pivot_data(df_daily.copy())


In [11]:
df_daily.columns

Index(['date', 'open_AGFS_0900', 'open_AGFS_0915', 'open_AGFS_0930',
       'open_AGFS_0945', 'open_AGFS_1000', 'open_AGFS_1015', 'open_AGFS_1030',
       'open_AGFS_1045', 'open_AGFS_1100',
       ...
       'EMA55_DOL$_1530', 'EMA55_DOL$_1545', 'EMA55_DOL$_1600',
       'EMA55_DOL$_1615', 'EMA55_DOL$_1630', 'EMA55_DOL$_1645',
       'EMA55_DOL$_1700', 'EMA55_DOL$_1715', 'EMA55_DOL$_1730',
       'EMA55_DOL$_1745'],
      dtype='object', length=6049)

In [12]:
df_daily_target.reset_index(inplace=True)

In [13]:
df_daily_target.columns

Index(['day', 'open_BGI$', 'close_BGI$', 'close_price_target',
       'open_price_target', 'behavior_target'],
      dtype='object')

In [14]:
df_daily = df_daily.merge(
    df_daily_target[['day', 'close_price_target', 'open_price_target', 'behavior_target']],
    left_on='date', right_on='day', how='inner'
)

In [15]:
df_daily.dropna(inplace=True)

In [16]:
df_daily

,date,open_AGFS_0900,open_AGFS_0915,open_AGFS_0930,open_AGFS_0945,open_AGFS_1000,open_AGFS_1015,open_AGFS_1030,open_AGFS_1045,open_AGFS_1100,...,EMA55_DOL$_1630,EMA55_DOL$_1645,EMA55_DOL$_1700,EMA55_DOL$_1715,EMA55_DOL$_1730,EMA55_DOL$_1745,day,close_price_target,open_price_target,behavior_target
0,2022-06-02,1767.0,1767.0,1767.0,1767.0,1764.0,1776.0,1778.0,1778.0,1766.0,...,5572.947718,5572.937799,5573.093235,5573.304976,5573.406049,5573.895333,2022-06-02,332.23,325.07,1
1,2022-06-03,1778.0,1778.0,1778.0,1778.0,1779.0,1766.0,1767.0,1762.0,1768.0,...,5575.844988,5575.319167,5574.915232,5574.525724,5574.150126,5573.581729,2022-06-03,331.88,331.88,0
2,2022-06-06,1753.0,1753.0,1753.0,1753.0,1752.0,1752.0,1753.0,1752.0,1750.0,...,5574.033019,5574.211197,5574.486119,5574.709971,5574.657758,5574.834267,2022-06-06,330.89,331.49,0
3,2022-06-07,1736.0,1736.0,1736.0,1736.0,1735.0,1723.0,1720.0,1720.0,1716.0,...,5640.188419,5641.179726,5642.135628,5642.768677,5643.482224,5644.190895,2022-06-07,330.80,331.88,0
4,2022-06-08,1712.0,1712.0,1712.0,1712.0,1713.0,1704.0,1707.0,1715.0,1716.0,...,5665.676277,5666.334696,5667.072707,5667.701896,5668.432328,5669.384138,2022-06-08,332.82,331.34,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,2024-11-13,1620.0,1620.0,1620.0,1620.0,1621.0,1619.0,1618.0,1610.0,1613.0,...,5824.151211,5824.413097,5824.863022,5825.189200,5825.557585,5825.930743,2024-11-13,346.16,348.15,1
616,2024-11-14,1618.0,1618.0,1618.0,1618.0,1618.0,1628.0,1616.0,1620.0,1624.0,...,5822.550148,5822.635929,5823.095503,5823.179771,5823.530172,5823.868058,2024-11-14,351.26,346.77,1
617,2024-11-18,1617.0,1617.0,1617.0,1617.0,1615.0,1609.0,1613.0,1609.0,1613.0,...,5800.004701,5799.442141,5798.774064,5798.183705,5797.901537,5797.521767,2024-11-18,347.02,351.82,0
618,2024-11-19,1601.0,1601.0,1601.0,1601.0,1601.0,1603.0,1600.0,1603.0,1606.0,...,5803.930075,5803.837465,5803.712270,5803.645367,5803.670569,5803.820513,2024-11-19,350.96,350.09,1


In [17]:
save_table(df_daily.head(6), title = 'Exemplo do Target diário para o fechamento, abertura e comportamento do mercado')
save_table(df_timestamp.head(6), title = 'Exemplo do Target timestamp para o fechamento, abertura e comportamento do mercado')

Table saved as CSV: ..\results\tables\csv\Tabela_3_Exemplo do Target diário para o fechamento, abertura e comportamento do mercado.csv
Table saved as CSV: ..\results\tables\csv\Tabela_2_Exemplo do Target timestamp para o fechamento, abertura e comportamento do mercado.csv


In [18]:
os.makedirs(output_dir, exist_ok=True)
df_daily_path = os.path.join(output_dir, 'df_daily.parquet').replace("/", "\\")
df_timestamp_path = os.path.join(output_dir, 'df_timestamp.parquet').replace("/", "\\")

# Salvando os DataFrames
df_daily.to_parquet(df_daily_path)
df_timestamp.to_parquet(df_timestamp_path)

# Print para confirmar os caminhos
print(f"Data saved to: {df_daily_path}")
print(f"Data saved to: {df_timestamp_path}")

Data saved to: ..\data\processed_data\df_daily.parquet
Data saved to: ..\data\processed_data\df_timestamp.parquet
